In [1]:
import tensorflow as tf
import scipy.io as sio
import matplotlib.pyplot as plt
import os
import numpy as np
import shutil
from datetime import datetime
now = datetime.now()

shutil.rmtree('D:/MATLAB/OSU/OSU-automapping/Train/log/', ignore_errors=True)

sess = tf.InteractiveSession()

In [2]:
learning_rate = 1e-4
channels_input = 32
channels_output = 32

fc_input_dim = int(128*128*channels_output/4**4)
fc_hidden_dim = 128

fc_input_dim

2048

In [3]:
def conv2d(x, W, dy=1 ,dx=1):
    return tf.nn.conv2d(x, W, strides=[1, dx, dy, 1], padding='SAME')

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
#     with tf.name_scope('summaries'):
#         mean = tf.reduce_mean(var)
#         tf.summary.scalar('mean', mean)
#         with tf.name_scope('stddev'):
#             stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
#         tf.summary.scalar('stddev', stddev)
#         tf.summary.scalar('max', tf.reduce_max(var))
#         tf.summary.scalar('min', tf.reduce_min(var))
#         tf.summary.histogram('histogram', var)

def conv_layer(input_tensor, filter_height, filter_width, channels_input, channels_output, layer_name, act=tf.nn.relu):
  # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
    # This Variable will hold the state of the weights for the layer
        with tf.name_scope('weights'):
            weights = weight_variable([filter_height, filter_width, channels_input, channels_output])
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_variable([channels_output])
            variable_summaries(biases)
        with tf.name_scope('Wx_plus_b'):
            preactivate = conv2d(input_tensor, weights,2,2) + biases
#             tf.summary.histogram('pre_activations', preactivate)
        activations = act(preactivate, name='activation')
#         tf.summary.histogram('activations', activations)
        return activations

def fc_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
    # This Variable will hold the state of the weights for the layer
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor, weights) + biases
#             tf.summary.histogram('pre_activations', preactivate)
        activations = act(preactivate, name='activation')
#         tf.summary.histogram('activations', activations)
        return activations
    
def output_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.softmax):
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
    # This Variable will hold the state of the weights for the layer
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor, weights) + biases
#             tf.summary.histogram('pre_activations', preactivate)
        activations = act(preactivate, name='activation')
#         tf.summary.histogram('activations', activations)
        return activations 

def batch_norm_layer(x, train_phase, scope_bn):
    with tf.variable_scope(scope_bn):
        beta = tf.Variable(tf.constant(0.0, shape=[x.shape[-1]]), name='beta', trainable=True)
        gamma = tf.Variable(tf.constant(1.0, shape=[x.shape[-1]]), name='gamma', trainable=True)
        axises = np.arange(len(x.shape) - 1)
        batch_mean, batch_var = tf.nn.moments(x, axises, name='moments')
        ema = tf.train.ExponentialMovingAverage(decay=0.5)

        def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean, batch_var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_var)

        mean, var = tf.cond(train_phase, mean_var_with_update, lambda: (ema.average(batch_mean), ema.average(batch_var)))
        normed = tf.nn.batch_normalization(x, mean, var, beta, gamma, 1e-3)
    return normed    
    
########################   Structure   ##############################

with tf.name_scope('spectrogram_input'):
    x = tf.placeholder("float", shape=[None, 128*128] , name='flat_input')
with tf.name_scope('target'):
    y_ = tf.placeholder("float", shape=[None, 2] , name='target')

with tf.name_scope('input'):
    x_image = tf.reshape(x, [-1,128,128,1] , name='2D_input')

h_conv1 = conv_layer(x_image, 16, 4, 1,              channels_output, 'conv_layer_1' , act=tf.nn.relu)
h_conv2 = conv_layer(h_conv1, 4, 16, channels_input, channels_output, 'conv_layer_2' , act=tf.nn.relu)
h_conv3 = conv_layer(h_conv2, 16, 4, channels_input, channels_output, 'conv_layer_3' , act=tf.nn.relu)
h_conv4 = conv_layer(h_conv3, 4, 16, channels_input, channels_output, 'conv_layer_4' , act=tf.nn.relu)

with tf.name_scope('reshape'):
    h_before_fc = tf.reshape(h_conv4,[-1,fc_input_dim])

h_fc = fc_layer(h_before_fc, fc_input_dim, fc_hidden_dim, 'hidden_layer' , act=tf.nn.relu)

with tf.name_scope('dropout'):
    keep_prob = tf.placeholder("float")
    h_fc_drop = tf.nn.dropout(h_fc, keep_prob)




y = output_layer(h_fc_drop, fc_hidden_dim, 2, 'output_layer_1' , act=tf.nn.softmax)
# tf.summary.histogram('y',y)


with tf.name_scope('cross_entropy'):
    diff = -y_*tf.log(tf.clip_by_value(y,1e-20,1.0))
    cross_entropy = tf.reduce_mean(diff)

# tf.summary.histogram('cross_entropy_individual',diff)
tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

In [4]:
############################## Initialization & Read Data  #######################

summaries_dir = 'D:/MATLAB/OSU/OSU-automapping/Train/log/train'+now.strftime("%Y%m%d-%H%M%S") + "/"

merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(summaries_dir + '/train',sess.graph)
test_writer = tf.summary.FileWriter(summaries_dir + '/test')
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()


# Test Data

testData=sio.loadmat('C:/OSU/SongMat/Test/532522 SakiZ - osu!memories.mat')
inputTest=testData['input'].reshape(-1,128*128)
targetTest=testData['target'].reshape(-1,4)
targetTest2=np.zeros([targetTest.shape[0],2])
targetTest2[:,0]=targetTest[:,0]+targetTest[:,1]+targetTest[:,2]
targetTest2[:,1]=targetTest[:,3]

NTest=targetTest2.shape[0]

In [5]:
############################## Training  #######################

batch_size = 10
step_size = 100
dataList=os.listdir('C:/OSU/SongMat/Train/')
# acc=np.zeros([1000,1])

n = 1
total_steps = 0
while n < np.size(dataList)-18:
    n = n + 3
    inputOsu=np.empty([0,128*128])
    targetOsu=np.empty([0,4])
    for k in range(n,n+15):
        
        data=sio.loadmat('C:/OSU/SongMat/Train/'+dataList[k])
        inputOsu=np.append(inputOsu,data['input1'].reshape(-1,128*128),axis=0)
        targetOsu=np.append(targetOsu,data['target1'].reshape(-1,4),axis=0)
               
    targetOsu2 = np.zeros([targetOsu.shape[0],2]);
    targetOsu2[:,0] = targetOsu[:,0]+targetOsu[:,1]+targetOsu[:,2]
    targetOsu2[:,1] = targetOsu[:,3]
    
    N=inputOsu.shape[0]
    print(N)
    
    #shuffle the order
    order = np.arange(N-step_size)
    np.random.shuffle(order)
    orderAcc = np.arange(NTest)
    np.random.shuffle(orderAcc)
    
    for i in range(N-step_size):
        if i%step_size == 0:
            summary, _  = sess.run([merged, train_step], feed_dict={x: inputOsu[order[i:i+step_size],:], y_: targetOsu2[order[i:i+step_size],:], keep_prob: 0.5})
            test_writer.add_summary(summary, i + total_steps)
        else:
            summary = sess.run(merged, feed_dict={x: inputOsu[order[i:i+step_size],:], y_: targetOsu2[order[i:i+step_size],:], keep_prob: 0.5})
            train_writer.add_summary(summary, i + total_steps)
        if i%1000 == 0:
            accMemory = accuracy.eval(feed_dict={x: inputTest[orderAcc[0:500],:], y_: targetTest2[orderAcc[0:500],:], keep_prob: 1.0}) 
            print("step %d, memory accuracy %g" %(i, accMemory))
#             print(y.eval(feed_dict={x: inputTest[orderAcc[1122:1130],:], keep_prob: 1.0}))
            
    total_steps += i
    
    
    

17803
step 0, memory accuracy 0.55
step 1000, memory accuracy 0.574
step 2000, memory accuracy 0.602
step 3000, memory accuracy 0.568
step 4000, memory accuracy 0.588
step 5000, memory accuracy 0.606
step 6000, memory accuracy 0.628
step 7000, memory accuracy 0.64
step 8000, memory accuracy 0.63
step 9000, memory accuracy 0.66
step 10000, memory accuracy 0.654
step 11000, memory accuracy 0.646
step 12000, memory accuracy 0.632
step 13000, memory accuracy 0.682
step 14000, memory accuracy 0.692
step 15000, memory accuracy 0.676
step 16000, memory accuracy 0.678
step 17000, memory accuracy 0.696
17923
step 0, memory accuracy 0.736
step 1000, memory accuracy 0.716
step 2000, memory accuracy 0.73
step 3000, memory accuracy 0.738
step 4000, memory accuracy 0.71
step 5000, memory accuracy 0.732
step 6000, memory accuracy 0.758
step 7000, memory accuracy 0.732
step 8000, memory accuracy 0.74
step 9000, memory accuracy 0.756
step 10000, memory accuracy 0.756
step 11000, memory accuracy 0.75
st

step 12000, memory accuracy 0.804
step 13000, memory accuracy 0.796
step 14000, memory accuracy 0.798
step 15000, memory accuracy 0.79
step 16000, memory accuracy 0.8
step 17000, memory accuracy 0.796
step 18000, memory accuracy 0.8
step 19000, memory accuracy 0.79
step 20000, memory accuracy 0.802
step 21000, memory accuracy 0.806
step 22000, memory accuracy 0.8
step 23000, memory accuracy 0.808
step 24000, memory accuracy 0.81
step 25000, memory accuracy 0.804
step 26000, memory accuracy 0.804
23437
step 0, memory accuracy 0.834
step 1000, memory accuracy 0.832
step 2000, memory accuracy 0.84
step 3000, memory accuracy 0.842
step 4000, memory accuracy 0.842
step 5000, memory accuracy 0.82
step 6000, memory accuracy 0.834
step 7000, memory accuracy 0.822
step 8000, memory accuracy 0.826
step 9000, memory accuracy 0.81
step 10000, memory accuracy 0.83
step 11000, memory accuracy 0.826
step 12000, memory accuracy 0.822
step 13000, memory accuracy 0.834
step 14000, memory accuracy 0.822


In [6]:
save_path = saver.save(sess, "D:/OSU/results/save/2.6/",global_step=n)

In [7]:
############################## Save the output of new maps ##############################

dataList=os.listdir('C:/OSU/SongMat/Create/')
# inputOsuNew=np.empty([0,128*128]);
for i in range(np.size(dataList)):
    if not dataList[i][0]=='y':
        data=sio.loadmat('C:/OSU/SongMat/Create/'+dataList[i])
        print(data['osuFolder'])
        
        inputOsuNew=data['input'].reshape(-1,128*128)
        outputOsuNew=data['target']
        outputOsu2New=np.zeros([outputOsuNew.shape[0],2]);
        outputOsu2New[:,0]=outputOsuNew[:,0]+outputOsuNew[:,1]+outputOsuNew[:,2]
        outputOsu2New[:,1]=outputOsuNew[:,3]
        
        y_result=np.empty([0,2])
        
        step_size=200
        cursor=0
        print ("test accuracy %g"%accuracy.eval(feed_dict={x: inputOsuNew[-500:,:], y_: outputOsu2New[-500:,:], keep_prob: 1.0}))
        while cursor<inputOsuNew.shape[0]:
            if inputOsuNew[cursor:,:].shape[0]>step_size:
                y_result1=y.eval(feed_dict={x: inputOsuNew[cursor:(cursor+step_size):1,:], keep_prob: 1.0})
                y_result=np.append(y_result,y_result1,axis=0)
            else:
                y_result1=y.eval(feed_dict={x: inputOsuNew[cursor:(cursor+step_size):1,:], keep_prob: 1.0})
                y_result=np.append(y_result,y_result1,axis=0)
            cursor+=step_size
            
        data_save={'y2':y_result}
        sio.savemat('C:/OSU/SongMat/Create/y2_'+dataList[i],data_save)
        
        

['D:\\Program Files (x86)\\osu!\\Songs\\532522 SakiZ - osu!memories\\']
test accuracy 0.704
